In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv('data/huyen-binh-chanh/data_clear/mua-duong-noi-bo.csv')

In [13]:
df.head()

,Ngay_dang,Phuong,Quan,Thanh_pho,Gia_ban,Gia_m2
0,01/08/2025,Xã Phạm Văn Hai,Huyện Bình Chánh,TPHCM,3.550000e+10,3.547871e+06
1,23/04/2025,Xã Vĩnh Lộc B,Huyện Bình Chánh,TPHCM,6.500000e+08,2.241379e+07
2,17/10/2024,Xã Vĩnh Lộc A,Huyện Bình Chánh,TPHCM,1.080000e+09,1.350000e+07
3,21/09/2024,Xã Tân Kiên,Huyện Bình Chánh,TPHCM,4.900000e+09,2.333333e+07
4,21/09/2024,Xã Tân Kiên,Huyện Bình Chánh,TPHCM,2.850000e+09,4.453125e+07


In [4]:
df['Dien_tich_su_dung'] = df['Dien_tich_su_dung'].fillna(df['Dien_tich_dat'])

In [7]:
df['Gia_m2'] = df['Gia_ban'] / df['Dien_tich_su_dung']

In [ ]:
df.drop(columns=['Ma_BDS', 'So_phong_ngu', 'So_nha_tam', 'Duong', 'Dien_tich_su_dung', 'Dien_tich_dat', 'Chieu_dai', 'Chieu_rong', 'Gia_ban'], inplace=True)

In [39]:
# Biến giá trị của cột Ngay_dang thành dạng %m-%Y
df['Ngay_dang'] = pd.to_datetime(df['Ngay_dang'], format='%d/%m/%Y').dt.to_period('M')

In [41]:
res = df.groupby(['Phuong', 'Ngay_dang']).agg({'Gia_m2': 'mean'}).reset_index()
res.sort_values(by=['Ngay_dang'], inplace=True)
res.reset_index(drop=True, inplace=True)

In [49]:
res['Quan'] = df['Quan']
res['Thanh_pho'] = "Thành phố Hồ Chí Minh"

In [50]:
res.head()

,Phuong,Ngay_dang,Gia_m2,Quan,Thanh_pho
0,Xã Vĩnh Lộc A,2021-01,0.000000e+00,Huyện Bình Chánh,Thành phố Hồ Chí Minh
1,Xã Vĩnh Lộc A,2021-03,3.557692e+07,Huyện Bình Chánh,Thành phố Hồ Chí Minh
2,Xã Phạm Văn Hai,2021-03,3.250000e+07,Huyện Bình Chánh,Thành phố Hồ Chí Minh
3,Xã Hưng Long,2021-03,7.666667e+06,Huyện Bình Chánh,Thành phố Hồ Chí Minh
4,Xã Tân Quý Tây,2021-04,1.009259e+07,Huyện Bình Chánh,Thành phố Hồ Chí Minh


In [51]:
res = res.applymap(lambda x : x.strip() if isinstance(x, str) else x)

/tmp/ipykernel_20623/2672745266.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  res = res.applymap(lambda x : x.strip() if isinstance(x, str) else x)


In [52]:
result = {}
for _, row in res.iterrows():
    tp = row["Thanh_pho"]
    q = row["Quan"]
    p = row["Phuong"]
    d = row["Ngay_dang"]
    g = row["Gia_m2"]
    
    result.setdefault(tp, {}).setdefault(q, {}).setdefault(p, {})[str(d)] = g

import json
js_str = json.dumps(result, ensure_ascii=False, indent=4)
with open('data/huyen-binh-chanh/data_clear/mua-duong-noi-bo-gia-m2.json', 'w', encoding='utf-8') as f:
    f.write(js_str)